# Introduction

We recommend first reading the [documentation](https://docs.pumas.ai/dev/) which goes over how to download and install the module.

## Quick Start

We first explore an example from the `pumas_be` function documentation.

In [ ]:
ENV["COLUMNS"] = 130;
ENV["LINES"] = 30;

Let us load the module through

In [ ]:
using Pkg
Pkg.activate(joinpath(@__DIR__, "tutorials", "bioequivalence"))
using Bioequivalence

The first example comes from a two sequences, two periods, two formulations crossover design (`RT|TR`) from Schütz, Labes, and Fuglsang (2014).

In [ ]:
SLF2014 = Bioequivalence.testdata("SLF2014_8")
show(SLF2014)

The `id` variable specifyies the identifier for each subject.

The `sequence` must be a `String` with each `Character` representing a formulation.

For example, `"RTTR"` represents `R` formulation in the first period, `T` formulation in the second and third periods, and `R` formulation in the fourth period.

The `period` variable must be integers specifying the period based on the sequence.

For example,

In [ ]:
id = 1
period = 1
sequence = "TR"
endpoint = 168.407

means that the endpoint for subject with ID `1` in the first period when receiving the `T` formulation recorded the value 168.407.

Notice that the study design is fully characterized by the unique values of the `sequence` variable.

In order to perform the bioequivalence analysis we can pass the data to the `pumas_be` function.

In [ ]:
Crossover = pumas_be(SLF2014)
show(Crossover)

When passing only the data argument it will use the default values for all the other arguments.

For example, if the dataset had the variable indicating the subject ID as `subj` instead of `id`,

In [ ]:
using DataFrames
rename!(SLF2014, "id" => "subj")
try
    pumas_be(SLF2014)
catch err
    err
end

In this case we can pass the variable position (i.e., `1`) or the name of the variable as a `Symbol` to the function.

In [ ]:
Crossover = pumas_be(SLF2014, id = :subj)
show(Crossover)

## Examining the Results

The `BioequivalenceStudy` will show the study design as well as how many subjects were in each sequence.

For convenience it also shows the formulation and periods as well as how many there are for that design.

A coefficients table will also be showing the parameter estimates and related values for each comparison among each non-reference formulation and the reference formulation. Those will include the point estimate (`PE`), standard error (`SE`), the 90% confidence interval in the natural log scale (i.e., `lnLB`, `lnUB`), the geometric means ratio (`GMR`), and the 90% confidence intervals in the natural scale (i.e., `LB`, `UB`).

The average bioequivalence is assessed based on the lower and upper bounds of the 90% percent confidence interval. Based on the regulatory guidelines for the FDA, the confidence intervals are reported in percentage form with two decimal places. In other words, if the lower bound value is 0.85793 the percentage form would be 85.793% and be rounded down to 85.79%. If the upper bound value is 1.20522 the percentage form would be 120.522% and be rounded up to 120.53%. The Bioequivalence results will have correct percentage format in decimal form (e.g. 1.2053 for 120.53%). 

Besides the bioequivalence results one can query a number of properties.

We can access the information about the design

In [ ]:
show(Crossover.design)

The data used for the model

In [ ]:
show(Crossover.data)

Statistics about the data such as:

how many observations did the initial data had.

In [ ]:
show(Crossover.data_stats.total)

how many observations did the analysis actually used (e.g., total - count of missing values).

In [ ]:
show(Crossover.data_stats.used_for_analysis)

as well as summary statistics of the endpoint distribution by sequence, period or formulation.

In [ ]:
show(Crossover.data_stats.sequence)

In [ ]:
show(Crossover.data_stats.period)

In [ ]:
show(Crossover.data_stats.formulation)

One can also query the model used for inference.
The exact API for extracting values from the models will depend on the design.
The most common API is documented [here](https://juliastats.org/StatsBase.jl/stable/statmodels/).
For example, we can obtain the coefficient of determination (`R²`) through the API.

In [ ]:
using StatsBase
r2(Crossover.model)

likewise we can obtain the `deviance` of the model through

In [ ]:
deviance(Crossover.model)

Derived statistics for the model such as the Wald test for join-significance for the sequence, period, and formulation covariates

In [ ]:
show(Crossover.model_stats.Wald)

and the linear regression mean estimates per formulation

In [ ]:
show(Crossover.model_stats.lsmeans)

are available as showen above.

You can also run models for various endpoints easily as following

In [ ]:
data = Bioequivalence.testdata("PJ2017_4_3")
show(data)

In [ ]:
auc = pumas_be(data)
cmax = pumas_be(data, endpoint = :Cmax)
output = Dict(endpoint => pumas_be(data, endpoint = endpoint) for endpoint in setdiff(propertynames(data), (:id, :sequence, :period)))

In [ ]:
show(output[:AUC])

In [ ]:
show(output[:Cmax])